# NB01: Study Overview

**Question:** What data do we have? What's broken? What's the rough performance landscape?

This notebook provides a high-level inventory of the `smart_retrieval_slm` study:
- Experiment health and completeness
- Coverage across models, datasets, agent types, and RAG components
- Overall performance landscape
- Metric correlation analysis (justifying F1 as primary metric)

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as scipy_stats

from analysis_utils import (
    load_all_results, setup_plotting, get_experiment_health_summary,
    print_search_space_summary, weighted_mean_with_ci,
    load_failed_experiments, analyze_failure_patterns,
    predict_context_length_issues,
    METRICS, PRIMARY_METRIC, MODEL_PARAMS, MODEL_TIER, BROKEN_MODELS,
)

setup_plotting()
STUDY_PATH = Path("../outputs/smart_retrieval_slm")

df = load_all_results(STUDY_PATH)
print(f"Loaded {len(df)} experiments")

## 1. Study Health

In [ ]:
health = get_experiment_health_summary(STUDY_PATH)
print("Study Health")
print("=" * 50)
for k, v in health.items():
    print(f"  {k:<22s}: {v}")

# Check for missing metrics
print("\nMetric availability:")
for m in METRICS:
    if m in df.columns:
        n_valid = df[m].notna().sum()
        print(f"  {m:<15s}: {n_valid:>4d}/{len(df)} ({n_valid/len(df)*100:.0f}%)")

# Flag broken models
print("\nBroken models (near-zero F1):")
for model in sorted(BROKEN_MODELS):
    subset = df[df['model_short'] == model]
    if len(subset) > 0 and PRIMARY_METRIC in subset.columns:
        mean_f1 = subset[PRIMARY_METRIC].mean()
        print(f"  {model}: mean F1 = {mean_f1:.4f} ({len(subset)} experiments)")
    else:
        print(f"  {model}: no experiments loaded")

## 1b. Experiment Failure Analysis

Which configurations crash? Are failures random or systematic?
Understanding failure patterns avoids wasting compute on doomed configs.

In [ ]:
# Load failed experiments
failed_df = load_failed_experiments(STUDY_PATH)
n_failed = len(failed_df)
n_total = len(df) + n_failed
fail_rate = n_failed / n_total * 100 if n_total > 0 else 0

print(f"Experiment Failure Analysis")
print("=" * 60)
print(f"  Completed:  {len(df):>4d}")
print(f"  Failed:     {n_failed:>4d}")
print(f"  Total:      {n_total:>4d}")
print(f"  Fail rate:  {fail_rate:.1f}%")

if not failed_df.empty:
    patterns = analyze_failure_patterns(failed_df)

    # Error category breakdown
    print(f"\nError Categories:")
    if 'by_error_type' in patterns:
        for cat, count in patterns['by_error_type'].items():
            pct = count / n_failed * 100
            bar = '#' * int(pct / 2)
            print(f"  {cat:<25s}: {count:>3d} ({pct:.0f}%) {bar}")

    # Failure rate by model
    print(f"\nFailure Rate by Model:")
    if 'by_model' in patterns:
        model_counts = patterns['by_model']
        for model, fail_count in sorted(model_counts.items(), key=lambda x: -x[1]):
            total_model = len(df[df['model_short'] == model]) + fail_count
            rate = fail_count / total_model * 100 if total_model > 0 else 0
            print(f"  {model:<20s}: {fail_count:>3d} failed / {total_model:>3d} total ({rate:.0f}%)")

    # Failure rate by retriever
    print(f"\nFailure Rate by Retriever:")
    if 'by_retriever' in patterns:
        retr_counts = patterns['by_retriever']
        for retr, fail_count in sorted(retr_counts.items(), key=lambda x: -x[1]):
            total_retr = len(df[df['retriever_type'] == retr]) + fail_count
            rate = fail_count / total_retr * 100 if total_retr > 0 else 0
            print(f"  {retr:<20s}: {fail_count:>3d} failed / {total_retr:>3d} total ({rate:.0f}%)")

    # Visualization: failure heatmap (model x retriever)
    if 'model_short' in failed_df.columns and 'retriever_type' in failed_df.columns:
        fail_ct = pd.crosstab(failed_df['model_short'], failed_df['retriever_type'])
        if fail_ct.size > 1:
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))

            sns.heatmap(fail_ct, annot=True, fmt='d', cmap='OrRd', ax=axes[0],
                        linewidths=0.5)
            axes[0].set_title('Failure Count: Model x Retriever')

            # Failure RATE heatmap (failures / total attempted)
            # Build total attempted from successes + failures
            success_ct = pd.crosstab(df['model_short'], df['retriever_type'])
            total_ct = success_ct.add(fail_ct, fill_value=0)
            rate_ct = fail_ct.div(total_ct).fillna(0) * 100
            sns.heatmap(rate_ct, annot=True, fmt='.0f', cmap='OrRd', ax=axes[1],
                        linewidths=0.5, vmin=0, vmax=100)
            axes[1].set_title('Failure Rate (%): Model x Retriever')

            plt.tight_layout()
            plt.show()

    # Context length risk prediction
    risky = predict_context_length_issues(STUDY_PATH)
    if isinstance(risky, pd.DataFrame) and not risky.empty:
        print(f"\nContext-Length Risk Predictions:")
        print(f"  {len(risky)} configurations flagged as high-risk")
        display(risky.head(10))
    else:
        print(f"\nContext-Length Risk: No high-risk configurations detected.")

    # Top-K and failure: do higher top_k values cause more failures?
    if 'top_k' in failed_df.columns and failed_df['top_k'].notna().sum() > 0:
        print(f"\nFailure Rate by Top-K:")
        for tk in sorted(failed_df['top_k'].dropna().unique()):
            tk_fail = len(failed_df[failed_df['top_k'] == tk])
            tk_success = len(df[df['top_k'] == tk]) if 'top_k' in df.columns else 0
            tk_total = tk_fail + tk_success
            rate = tk_fail / tk_total * 100 if tk_total > 0 else 0
            print(f"  top_k={int(tk):<4d}: {tk_fail:>3d} failed / {tk_total:>3d} total ({rate:.0f}%)")
else:
    print("\nNo failed experiments found — all experiments completed successfully.")

## 2. Coverage

In [ ]:
print_search_space_summary(df)

# Model x Dataset crosstab heatmap
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Model x Dataset
ct_model_ds = pd.crosstab(df['model_short'], df['dataset'])
sns.heatmap(ct_model_ds, annot=True, fmt='d', cmap='YlGn', ax=axes[0])
axes[0].set_title('Experiments: Model x Dataset')

# Agent type x Dataset
ct_agent_ds = pd.crosstab(df['agent_type'], df['dataset'])
sns.heatmap(ct_agent_ds, annot=True, fmt='d', cmap='YlOrRd', ax=axes[1])
axes[1].set_title('Experiments: Agent Type x Dataset')

plt.tight_layout()
plt.show()

## 3. Performance Landscape

In [ ]:
# Filter out broken models for performance analysis
df_clean = df[~df['model_short'].isin(BROKEN_MODELS)].copy()
print(f"After filtering broken models: {len(df_clean)} experiments")

# Model leaderboard
leaderboard = weighted_mean_with_ci(df_clean, 'model_short', PRIMARY_METRIC)
print("\nModel Leaderboard (F1):")
display(leaderboard)

# Bar chart with CI
fig, ax = plt.subplots(figsize=(12, 5))
x = range(len(leaderboard))
yerr_low = leaderboard['mean'] - leaderboard['ci_low']
yerr_high = leaderboard['ci_high'] - leaderboard['mean']
colors = [sns.color_palette('husl', len(leaderboard))[i] for i in range(len(leaderboard))]
ax.bar(x, leaderboard['mean'], yerr=[yerr_low, yerr_high], capsize=5,
       color=colors, alpha=0.8, edgecolor='black', linewidth=0.5)
ax.set_xticks(x)
ax.set_xticklabels(leaderboard['model_short'], rotation=30, ha='right')
ax.set_ylabel('Mean F1')
ax.set_title('Model Leaderboard — Mean F1 with 95% CI')
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Dataset violin plots of F1 distribution
fig, ax = plt.subplots(figsize=(10, 5))
datasets = sorted(df_clean['dataset'].unique())
data_for_violin = [df_clean[df_clean['dataset'] == ds][PRIMARY_METRIC].dropna().values
                   for ds in datasets]
parts = ax.violinplot(data_for_violin, showmeans=True, showmedians=True)
ax.set_xticks(range(1, len(datasets) + 1))
ax.set_xticklabels(datasets)
ax.set_ylabel('F1')
ax.set_title('F1 Distribution by Dataset')
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Metric Correlation

Justification for using F1 as the primary metric: if F1, exact_match, bertscore, and bleurt are highly correlated, conclusions drawn from F1 generalize.

In [ ]:
# Compute Spearman correlations across all available metrics
available = [m for m in METRICS if m in df_clean.columns and df_clean[m].notna().sum() >= 10]
print(f"Metrics with sufficient data: {available}")

if len(available) >= 2:
    metric_df = df_clean[available].dropna()
    corr_matrix = metric_df.corr(method='spearman')

    fig, ax = plt.subplots(figsize=(8, 6))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdYlGn',
                vmin=-1, vmax=1, mask=mask, ax=ax, square=True)
    ax.set_title('Spearman Correlation Between Metrics')
    plt.tight_layout()
    plt.show()

    # Print pairwise correlations with F1
    print("\nCorrelation with F1:")
    for m in available:
        if m != PRIMARY_METRIC:
            rho, pval = scipy_stats.spearmanr(
                metric_df[PRIMARY_METRIC], metric_df[m]
            )
            print(f"  {m:<15s}: rho={rho:.3f}, p={pval:.2e}")
else:
    print("Not enough metrics with data for correlation analysis.")

## 5. Summary

Key takeaways from this overview — fill in after running on real data.

In [ ]:
print("STUDY OVERVIEW SUMMARY")
print("=" * 60)
print(f"Total experiments (loaded):   {len(df)}")
print(f"After broken-model filter:    {len(df_clean)}")
print(f"Models:                       {sorted(df['model_short'].unique())}")
print(f"Datasets:                     {sorted(df['dataset'].unique())}")
print(f"Agent types:                  {sorted(df['agent_type'].unique())}")
print(f"Experiment types:             {df['exp_type'].value_counts().to_dict()}")
if PRIMARY_METRIC in df_clean.columns:
    print(f"\nOverall F1 (clean):  mean={df_clean[PRIMARY_METRIC].mean():.4f}, "
          f"std={df_clean[PRIMARY_METRIC].std():.4f}")